# 微调步骤
- 加载数据集
- 对数据集进行预处理
- 加载模型
- 使用数据集对模型微调
- 保存模型
-

In [1]:
from datasets import  load_dataset
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

print(dataset["train"])


/home/gengzi/anaconda3/envs/env-vllm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since yelp_review_full couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'yelp_review_full' at /home/gengzi/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/0.0.0/c1f9ee939b7d05667af864ee1cb066393154bf85 (last modified on Tue Feb 11 21:28:37 2025).


Dataset({
    features: ['label', 'text'],
    num_rows: 650000
})


In [22]:
from  transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")


dataset_1000 = dataset["train"].shuffle(seed=55).select(range(1000))
dataset_1000_test = dataset["test"].shuffle(seed=55).select(range(1000))

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets_train = dataset_1000.map(tokenize_function,batched=True)

tokenized_datasets_test = dataset_1000_test.map(tokenize_function,batched=True)










Map: 100%|██████████| 1000/1000 [00:00<00:00, 3414.62 examples/s]


In [23]:
print(tokenized_datasets_train)
print(tokenized_datasets_test)
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

print(tokenized_datasets_train[310])
print(tokenized_datasets_test[100])

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})
Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})
{'label': 4, 'text': 'If you are ever in Vegas you MUST try this place. They have coconut mojitos that are to die for! Their sushi tastes so fresh every time. It is more of a simple styled sushi restaurant but it is by far one of my favorites for lunch or afternoon pick-me-up. This place will not disappoint and the wait staff is incredibly friendly.', 'input_ids': [101, 1409, 1128, 1132, 1518, 1107, 6554, 1128, 150, 13329, 1942, 2222, 1142, 1282, 119, 1220, 1138, 20331, 182, 1186, 23324, 2155, 1115, 1132, 1106, 2939, 1111, 106, 2397, 28117, 5933, 18689, 1177, 4489, 1451, 1159, 119, 1135, 1110, 1167, 1104, 170, 3014, 14174, 28117, 5933, 4382, 1133, 1122, 1110, 1118, 1677, 1141, 1104, 1139, 25735, 1111, 5953, 1137, 4427, 3368, 118, 1143, 118, 1146, 119, 1188, 1282

In [25]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased",num_labels=5,torch_dtype="auto")


from transformers import TrainingArguments,Trainer
## 制定检查点的保存位置
train_args = TrainingArguments(output_dir= "/mnt/e/models/checkpoint")



train = Trainer(model=model,args=train_args,train_dataset=tokenized_datasets_train,eval_dataset=tokenized_datasets_test)


train.train()


train.save_model("/mnt/e/models/checkpoint/model")
tokenizer.save_pretrained("/mnt/e/models/checkpoint/model")




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


('/mnt/e/models/checkpoint/model/tokenizer_config.json',
 '/mnt/e/models/checkpoint/model/special_tokens_map.json',
 '/mnt/e/models/checkpoint/model/vocab.txt',
 '/mnt/e/models/checkpoint/model/added_tokens.json',
 '/mnt/e/models/checkpoint/model/tokenizer.json')

In [36]:
## 加载微调后的模型

from transformers import AutoModel,AutoTokenizer
import torch

old_model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased",num_labels=5,torch_dtype="auto",device_map="cuda")
old_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased",device_map="cuda")

inputs  = old_tokenizer(text="今天天气一般",return_tensors="pt").to(0)
# 进行推理
with torch.no_grad():
    outputs = old_model(**inputs)
# 获取 logits
logits = outputs.logits
# 获取预测的类别
predicted_class_id = torch.argmax(logits, dim=-1).item()
print(f"预测的类别 ID: {predicted_class_id}")














Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


预测的类别 ID: 3


In [37]:
new_model = AutoModelForSequenceClassification.from_pretrained("/mnt/e/models/checkpoint/model",num_labels=5,torch_dtype="auto",device_map="cuda")
new_tokenizer = AutoTokenizer.from_pretrained("/mnt/e/models/checkpoint/model",device_map="cuda")

new_inputs  = new_tokenizer(text="今天天气真好",return_tensors="pt").to(0)
# 进行推理
with torch.no_grad():
    outputs = old_model(**new_inputs)
# 获取 logits
logits = outputs.logits
# 获取预测的类别
predicted_class_id = torch.argmax(logits, dim=-1).item()
print(f"预测的类别 ID: {predicted_class_id}")


预测的类别 ID: 3


In [39]:
dataset_evaluate = tokenized_datasets_train.shuffle(seed=55).select(range(100))
train.evaluate(dataset_evaluate)

{'eval_loss': 0.5872434377670288,
 'eval_runtime': 1.3534,
 'eval_samples_per_second': 73.89,
 'eval_steps_per_second': 9.606,
 'epoch': 3.0}